In [21]:
import json
from pathlib import Path
import os
import pprint
class Error:
    def __init__(self, assetId, field, expectedValueRange, actualValue, criticality, timestamp):
        self.assetId = assetId
        self.field = field
        self.expectedValueRange = expectedValueRange
        self.actualValue = actualValue
        self.criticality = criticality
        self.timestamp = timestamp
    def __str__(self):
        return "Test failled for asset id " + str(self.assetId) + ". Expected value for field " + self.field + " was in range " + str(self.expectedValueRange) + ", received value " + str(self.actualValue) + " at " + self.timestamp
        
class Detectors:
    def __init__(self):
        self.missions = []
        self.sites = []
        self.assets = []
        self.load_config()
        self.load_assets()
        self.connect_to_carbon()
        
    def load_config(self):
        data = None
        with open('./config.json') as f:
          data = json.load(f)
        self.missions = data["missions"]
        self.sites = data["sites"]
        
    def load_assets(self):        
        p = Path('./assets')
        for child in p.iterdir():
            filename, ext = os.path.splitext(str(child))
            if(ext == ".json"):
                with open(str(child)) as f:
                    self.assets.append(json.load(f))
                    
    def connect_to_carbon(self):
        self.carbon = {}
    
    def within(self, val, v_range):
        return val >= v_range[0] and val <= v_range[1]
                    
    def getLatestData(self, id):
        # TODO
        return 100
    
    def runTests(self):
        errors = []
        for asset in self.assets:
#             data = self.getLatestData(asset["id"])
            #
            #
            # THIS Is JUST BECAUSE WE DON't ACTUALLY HAVE A DATABASE OR DATA YET
            data = {}
            for field in asset["fields"]:
                data[field] = 9999999
            data["timestamp"] = "12:00:00 00:00:00"
            #
            #
            #
            
            
            for test in asset["tests"]:
                # IF conditions must be met for a test to be accurate
                if len(test["if"]) > 0:
                    conditions_met = True
                    for condition in test["if"]:
                        if not self.within(data[condition["field"]],condition["value_range"]):
                            conditions_met = False
                    
                    if conditions_met or not conditions_met:
                        for condition in test["then_conditions"]:
                            if not self.within(data[condition["field"]], condition["value_range"]):
                                errors.append(Error(asset["id"], condition["field"], condition["value_range"], data[condition["field"]], test["test_classification"], data["timestamp"]))
                            
                    else:
                        # If the conditions are not met, the test is not run
                        0
                
                # If there are no prerequisits to running the test
                else:
                    for condition in test["then_conditions"]:
                        if not self.within(data[condition["field"]], condition["value_range"]):
                            errors.append(Error(asset["id"], condition["field"], condition["value_range"], data[condition["field"]], test["test_classification"], data["timestamp"]))
        return errors
    
    def print(self):
        pp = pprint.PrettyPrinter(indent=4)
        print("\n--- Missions ---")
        pp.pprint(self.missions)
        print("\n--- Sites ---")
        pp.pprint(self.sites)
        print("\n--- Assets ---")
        pp.pprint(self.assets)
        

a = Detectors()
for e in a.runTests():
    print(e)

Test failled for asset id 1. Expected value for field connection was in range [1, 1], received value 9999999 at 12:00:00 00:00:00
Test failled for asset id 1. Expected value for field fan_speed was in range [0, 300], received value 9999999 at 12:00:00 00:00:00
Test failled for asset id 0. Expected value for field connection was in range [1, 1], received value 9999999 at 12:00:00 00:00:00
Test failled for asset id 0. Expected value for field temperature was in range [20, 130], received value 9999999 at 12:00:00 00:00:00
Test failled for asset id 2. Expected value for field connection was in range [1, 1], received value 9999999 at 12:00:00 00:00:00
Test failled for asset id 2. Expected value for field fan_speed was in range [0, 300], received value 9999999 at 12:00:00 00:00:00


#### Once we have the errors, this is probably when we want to call carbon